In [1]:
category = 6

In [2]:
import pandas as pd 
import numpy as np
import preprocessor as p
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.corpus import stopwords
#nltk.download('stopwords')

import pandas as pd

df = pd.read_csv(f"/mnt/smdata/twitter/VAMoS/finalData/ALLcat{category}.csv", engine='python')
print(df.shape)

#removing stopwords
all_languages = ['arabic', 'azerbaijani', 'danish', 'dutch', 'english', 'finnish', 'french', 'german', 'greek','hungarian', 'indonesian', 
 'italian', 'kazakh', 'nepali', 'norwegian', 'portuguese', 'romanian', 'russian', 'slovene', 'spanish', 'swedish', 'tajik', 'turkish']

stops = set(stopwords.words(all_languages))
print("total stopwords in all languages = "+ str(len(list(stops))))

vectorizer_model = CountVectorizer(analyzer = 'word', stop_words=list(stops))

df = df.drop_duplicates(subset=['text'])
print("after removing duplicate tweets:")
print(df.shape)
#convert to list 
tweets = df.text.to_list()
print("total tweets = "+str(len(tweets)))

# remove URLs, mentions, emojis, RT etc.
p.set_options(p.OPT.URL, p.OPT.EMOJI, p.OPT.MENTION, p.OPT.RESERVED, p.OPT.SMILEY)

cleaned_tweets = []
for i in tweets:
    ct = p.clean(i)
    cleaned_tweets.append(ct)

seen = set()
unique_tweets = []
for item in cleaned_tweets:
    if item not in seen:
        seen.add(item)
        unique_tweets.append(item)

print("total unique preprocessed tweets = "+str(len(unique_tweets)))

"""
k=1
for tweet in cleaned_tweets:
    print(k)
    print(tweet)
    k=k+1
"""

/home/ubuntu/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-04-11 02:58:17.221762: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-11 02:58:19.611310: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-11 02:58:19.612009: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-11 02:58:21.421273: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


(442, 3)
total stopwords in all languages = 6715
after removing duplicate tweets:
(265, 3)
total tweets = 265
total unique preprocessed tweets = 264


'\nk=1\nfor tweet in cleaned_tweets:\n    print(k)\n    print(tweet)\n    k=k+1\n'

In [3]:
# language detection
from langdetect import detect

eng_esp = []
for tweet in unique_tweets:
    try:
        if len(tweet)>0 and (detect(tweet)=='en' or detect(tweet)=='es'):
            eng_esp.append(tweet)
    except:
        continue

print("eng/esp tweets = "+str(len(eng_esp)))

eng/esp tweets = 238


In [4]:
#geolocation using mordecai that uses docker in the background
# -> sudo docker container ls
# -> sudo docker stop <...>
# ->  sudo docker .... see mordecai github repo
import spacy
nlp = spacy.load("/home/ubuntu/.local/lib/python3.8/site-packages/en_core_web_lg/en_core_web_lg-2.3.1/")

from mordecai import Geoparser
geo = Geoparser()

state = []
country = []

for i in range(0,len(eng_esp)):
    if i%1000 == 0:
        print("<------------------"+str(i)+"------------------>")
    a = geo.geoparse(eng_esp[i])
    for j in range(0,len(a)):
        if a[j]['country_conf'] > 0.75:
            #print(a)
            print(str(a[j]["geo"]["admin1"]) + " : " + str(a[j]["geo"]["country_code3"]))
            state.append(a[j]["geo"]["admin1"])
            country.append(a[j]["geo"]["country_code3"])

sk = [('state',state),('country',country)]
data = dict(sk)
df = pd.DataFrame(data)
df.to_csv(f"topicModels/eng_esp_GEO_cat{category}.csv",index=False)
print("saved location data to csv")
            

Models path: /home/ubuntu/.local/lib/python3.8/site-packages/mordecai/models/
<------------------0------------------>
1/1 [==============================] - 0s 14ms/step
Atlántico : COL
1/1 [==============================] - 0s 22ms/step
Piedmont : ITA
1/1 [==============================] - 0s 21ms/step
Jalisco : MEX
1/1 [==============================] - 0s 19ms/step
NA : MEX
1/1 [==============================] - 0s 24ms/step
NA : VCT
1/1 [==============================] - 0s 20ms/step
Atlántico : COL
1/1 [==============================] - 0s 15ms/step
Campania : ITA
1/1 [==============================] - 0s 14ms/step
North Dakota : USA
Quebec : CAN
Oio : GNB
1/1 [==============================] - 0s 19ms/step
Moravskoslezský : CZE
1/1 [==============================] - 0s 17ms/step
Seoul : KOR
1/1 [==============================] - 0s 24ms/step
saved location data to csv


In [53]:
topic_model = BERTopic(nr_topics="auto", vectorizer_model=vectorizer_model, language="multilingual")
topics, probabilities = topic_model.fit_transform(eng_esp)

tot_topics = len(topic_model.get_topics()) - 1
print("total topics = " + str(tot_topics))
topic_model.get_topic_freq().head(11)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

,Topic,Count
0,-1,25430
1,0,2266
2,1,1343
3,2,1271
4,3,1220
5,4,661
6,5,500
7,6,485
8,7,456
9,8,385


In [54]:
topic_model.visualize_barchart(top_n_topics=12,n_words=10,title=f"Cat {category} (# eng/esp tweets = {len(eng_esp)}) : Topic word scores (total = {tot_topics})")

In [55]:
topic_model.save(f"/mnt/smdata/twitter/VAMoS/topicModels/eng_esp_cat{category}_model")
print("model saved")
df = topic_model.get_document_info(eng_esp)
df.to_csv(f"topicModels/eng_esp_topicinfo_cat{category}.csv")
print("doc info saved")

model saved
doc info saved


In [56]:
#reduce the number of topics after training the model. This is a great option if retraining the model will take many hours
#new_topics, new_probs = topic_model.reduce_topics(cleaned_tweets, nr_topics=250)

In [57]:
#!pip install -U kaleido

In [58]:
#!pip install langdetect

In [59]:
#from langdetect import detect
#if detect("War doesn't show who's right, just who's left.")=='en':
#    print("english")